In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
# data is preprocessed by taking 64x64 crops around each digit sequence

# train baseline character seqeunce CNN model

# CNN with 11 hiddenlayers

# five independent softmax classifiers

# # each classifier includes null-character output to model variable length seq
MODELPATH = '/content/gdrive/My Drive/Colab Notebooks/yai7_CV1/modelsave/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
# ST-CNN single

# ST-CNN multi

In [7]:
# localization network 

# two layer fully connected network with 32 units per layer

In [8]:
# trained from scratch with SGD and dropout
# batchj size 128
# 400k iterations
# learning rate of 0.01 decrease 1/10 every 80k iter
# weight decay 0.0005
# dropout 0.5 for all layters except first conv layer and localization network
# learning rate for Spatial transformer network = 1/10 of base learning rate


# random initialized weights

# regression layer of spatial transformer are intitialized to predict identity transform

In [9]:
#conv[48,5,1,2]-max[2]-
#conv[64,5,1,2]-conv[128,5,1,2]-max[2]-conv[160,5,1,2]-conv[192,5,1,2]-max[2]-conv[192,5,1,2]-
#conv[192,5,1,2]-max[2]-conv[192,5,1,2]-fc[3072]-fc[3072]-fc[3072]]

# with ReLU after each layer

# followed by 5 parallel fc + softmax for classification


# ST single
# conv[32,5,1,2]-max[2]-conv[32,5,1,2]-fc[32]-fc[32]
# vs ST multi
# fc[32]-fc[32]


In [10]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Feb 15 12:04:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [12]:
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

In [13]:
import torchvision.transforms as T

import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
"""
train_set = torchvision.datasets.SVHN(
    root = './data/SVHN',
    split = 'train',
    download = True,
    transform = T.ToTensor()
)
test_set = torchvision.datasets.SVHN(
    root = './data/SVHN',
    split = 'test',
    download = True,
    transform = T.ToTensor()
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1)
"""

In [9]:
"""
mean = 0.
std = 0.
nb_samples = 0.

for data, _ in train_loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(f'mean = {mean}')
print(f'std = {std}')
"""
# mean = tensor([0.4377, 0.4438, 0.4728])
# std = tensor([0.1201, 0.1231, 0.1052])

mean = tensor([0.4377, 0.4438, 0.4728])
std = tensor([0.1201, 0.1231, 0.1052])


In [14]:
transforms = T.Compose([
                        T.ToTensor(),
                        T.Normalize((0.4377, 0.4438, 0.4728), (0.1201, 0.1231, 0.1052))
])

train_set = torchvision.datasets.SVHN(
    root = './data/SVHN',
    split = 'train',
    download = True,
    transform = T.ToTensor()
)
test_set = torchvision.datasets.SVHN(
    root = './data/SVHN',
    split = 'test',
    download = True,
    transform = T.ToTensor()
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32)

Using downloaded and verified file: ./data/SVHN/train_32x32.mat
Using downloaded and verified file: ./data/SVHN/test_32x32.mat


In [15]:
#conv[48,5,1,2]-max[2]-
#conv[64,5,1,2]-conv[128,5,1,2]-max[2]-conv[160,5,1,2]-conv[192,5,1,2]-max[2]-conv[192,5,1,2]-
#conv[192,5,1,2]-max[2]-conv[192,5,1,2]-fc[3072]-fc[3072]-fc[3072]]

# with ReLU after each layer

# followed by 5 parallel fc + softmax for classification


# ST single
# conv[32,5,1,2]-max[2]-conv[32,5,1,2]-fc[32]-fc[32]
# vs ST multi
# fc[32]-fc[32]


# omit the first max pooling because image size is 64x64 --> 32x32

class ST_Single_CNN(nn.Module):
    def __init__(self):
        super(ST_Single_CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 48, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(48, 64, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2)
        self.conv4 = nn.Conv2d(128, 160, kernel_size=5, stride=1, padding=2)
        self.conv5 = nn.Conv2d(160, 192, kernel_size=5, stride=1, padding=2)
        self.conv6 = nn.Conv2d(192, 192, kernel_size=5, stride=1, padding=2)
        self.conv7 = nn.Conv2d(192, 192, kernel_size=5, stride=1, padding=2)
        self.conv8 = nn.Conv2d(192, 192, kernel_size=5, stride=1, padding=2)

        self.loc_conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.loc_conv2 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2)

        self.loc_fc1 = nn.Linear(8192,32)
        self.loc_fc2 = nn.Linear(32, 6)

        # initialize to predict identity transformer
        bias = torch.from_numpy(np.array([1,0,0,0,1,0]))
        nn.init.constant(self.loc_fc2.weight, 0)
        self.loc_fc2.bias.data.copy_(bias)

        self.fc1 = nn.Linear(3072, 3072)
        self.fc2 = nn.Linear(3072, 3072)

        self.fc_digit_length = nn.Linear(3072,7)

        self.fc_digit1 = nn.Linear(3072, 11)
        self.fc_digit2 = nn.Linear(3072, 11)
        self.fc_digit3 = nn.Linear(3072, 11)
        self.fc_digit4 = nn.Linear(3072, 11)
        self.fc_digit5 = nn.Linear(3072, 11)
        self.fc_digit6 = nn.Linear(3072, 11)

        self.dropout = nn.Dropout(p=0.5)

        self.maxpool = nn.MaxPool2d(2,2)

    def forward(self, x):
        batch_size = x.size(0)

        theta = self.loc_conv1(x)
        theta = F.relu(theta)
        # didnt omitt  max pooling here
        theta = self.maxpool(theta)
        theta = self.loc_conv2(theta)
        theta = F.relu(theta)

        theta = self.loc_fc1(theta.view(batch_size, -1))
        theta = F.relu(theta)
        theta = self.loc_fc2(theta).view(batch_size, 2, 3)

        grid = F.affine_grid(theta=theta, size=x.shape)
        x = F.grid_sample(x, grid)

        x = self.conv1(x)
        x = F.relu(x)
        # omitted max pooling here

        x = self.dropout(x)
        x = self.conv2(x)
        x = F.relu(x)

        x = self.dropout(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.maxpool(x)

        x = self.dropout(x)
        x = self.conv4(x)
        x = F.relu(x)

        x = self.dropout(x)
        x = self.conv5(x)
        x = F.relu(x)
        x = self.maxpool(x)

        x = self.dropout(x)
        x = self.conv6(x)
        x = F.relu(x)

        x = self.dropout(x)
        x = self.conv7(x)
        x = F.relu(x)
        x = self.maxpool(x)
        
        x = self.dropout(x)
        x = self.conv8(x)
        x = F.relu(x)

        x = x.view(batch_size, 3072)

        x = self.dropout(x)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)

        digit1 = self.fc_digit1(x)
        #digit2 = self.fc_digit2(x)
        #digit3 = self.fc_digit3(x)
        #digit4 = self.fc_digit4(x)
        #digit5 = self.fc_digit5(x)
        #digit6 = self.fc_digit6(x)
        #digit_length = self.fc_digit_length(x)

        #return [digit1, digit2, digit3, digit4, digit5, digit6], digit_length
        return digit1

# with reference to https://github.com/potterhsu/SVHNClassifier-PyTorch

In [33]:
# dropout 0.5 for all layters except first conv layer and localization network
# learning rate for Spatial transformer network = 1/10 of base learning rate


In [34]:
#conv[48,5,1,2]-max[2]-
#conv[64,5,1,2]-conv[128,5,1,2]-max[2]-conv[160,5,1,2]-conv[192,5,1,2]-max[2]-conv[192,5,1,2]-
#conv[192,5,1,2]-max[2]-conv[192,5,1,2]-fc[3072]-fc[3072]-fc[3072]]

In [35]:
# define custom loss function with cross_entropy

def custom_loss_function(label_length, label_digits, pred_length, pred_digits):
    loss = F.cross_entropy(label_length, pred_length)
    loss += F.cross_entropy(label_digits[0], pred_digits[0])

    if label_length > 1:
        loss += F.cross_entropy(label_digits[1], pred_digits[1])
    if label_length > 2:
        loss += F.cross_entropy(label_digits[2], pred_digits[2])
    if label_length > 3:
        loss +=F.cross_entropy(label_digits[3], pred_digits[3])
    if label_length > 4:
        loss += F.cross_entropy(label_digits[4], pred_digits[4])
    if label_length > 5:
        loss += F.cross_entropy(label_digits[5], pred_digits[5])
        
    return loss

In [36]:
# define custom loss function with cross_entropy

def custom_loss_function(label_length, label_digits, pred_length, pred_digits):
    legnth_ce = F.cross_entropy(label_length, pred_length)
    digit1_ce = F.cross_entropy(label_digits[0], pred_digits[0])
    digit2_ce = F.cross_entropy(label_digits[1], pred_digits[1])
    digit3_ce = F.cross_entropy(label_digits[2], pred_digits[2])
    digit4_ce = F.cross_entropy(label_digits[3], pred_digits[3])
    digit5_ce = F.cross_entropy(label_digits[4], pred_digits[4])
    digit6_ce = F.cross_entropy(label_digits[5], pred_digits[5])

    loss = length_ce + digit1_ce + digit2_ce + digit3_ce + digit4_ce + digit5_ce + digit6_ce
    return loss

In [37]:
# define custom loss function with cross_entropy

def custom_acc_function(label_length, label_digits, pred_length, pred_digits):
    legnth_check = abel_length == pred_length
    digit1_check = (label_digits[:,0] == pred_digits[:,0]).mean()
    digit2_check = (label_digits[:,1] == pred_digits[:,1]).mean()
    digit3_check = (label_digits[:,2] == pred_digits[:,2]).mean()
    digit4_check = (label_digits[:,3] == pred_digits[:,3]).mean()
    digit5_check = (label_digits[:,4] == pred_digits[:,4]).mean()
    digit6_check = (label_digits[:,5] == pred_digits[:,5]).mean()

    acc = length_check + digit1_check + digit2_check + digit3_check + digit4_check + digit5_check + digit6_check
    acc /= 7
    return acc

In [16]:
model = ST_Single_CNN()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


In [17]:
model.to(device)

ST_Single_CNN(
  (conv1): Conv2d(3, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(128, 160, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(160, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv6): Conv2d(192, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv7): Conv2d(192, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv8): Conv2d(192, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (loc_conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (loc_conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (loc_fc1): Linear(in_features=8192, out_features=32, bias=True)
  (loc_fc2): Linear(in_features=32, out_features=6, bias=True)
  (fc1): Linear(in_features=3072, out_features=3072, bias=True)
  (fc

In [24]:
local_list=['loc_conv1.weight', 'loc_conv1.bias',
            'loc_conv2.weight', 'loc_conv2.bias',
            'loc_fc1.weight', 'loc_fc1.bias',
            'loc_fc2.weight', 'loc_fc2.bias']
local_params=list(filter(lambda kv: kv[0] in local_list, model.named_parameters()))
base_params=list(filter(lambda kv: kv[0] not in local_list, model.named_parameters()))

local_params = [param[1] for param in local_params]
base_params = [param[1] for param in base_params]

In [19]:
def train_one_epoch(model, train_loader, loss_function, optimizer, epoch):
    model.train()
    length = len(train_loader)
    running_loss = 0.0

    for i, (image, label) in enumerate(train_loader):
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        pred = model(image)
        loss = loss_function(pred, label)
    
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()

        if i % 50 == 0:
            print(f'after looking at {(i+1)*128} images, running_loss is {running_loss/(i+1):.4f}')

    print(f'epoch {epoch} : loss {running_loss/length:.4f}')


def eval_one_epoch(model, test_loader, epoch, best_val_acc):
    model.eval()
    length = len(test_loader)
    running_acc = 0.0

    for i, (image, label) in enumerate(test_loader):
        image, label = image.to(device), label.numpy()

        pred = model(image)
        pred_label = pred.argmax(axis=1).detach().cpu().numpy()
        running_acc += (pred_label==label).mean()

        if i % 50 == 0:
            print(f'after looking at {(i+1)*32} images, running_acc is {running_acc/(i+1):.4f}')

    val_acc = running_acc/length

    print(f'epoch {epoch} : acc {val_acc:.4f}')

    if val_acc > best_val_acc:
        print("### saving current model...### \n")
        torch.save(model.state_dict(), MODELPATH+f'SVHN_pretrained_{epoch}_epochs')
        best_val_acc = val_acc

    return val_acc

In [20]:
# hyperparameters from paper
# assuming same for pretraining

learning_rate = 0.01
momentum = 0.9
weight_decay = 5e-4
epochs = 100

In [21]:
loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD([{'params':base_params},
                             {'params':local_params, 'lr':learning_rate/10}], 
                             lr=learning_rate, weight_decay=weight_decay)

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=20, gamma=0.1
)


In [25]:
val_acc_list = []
best_val_acc = 0

for epoch in range(epochs):

    local_list=['loc_conv1.weight', 'loc_conv1.bias',
            'loc_conv2.weight', 'loc_conv2.bias',
            'loc_fc1.weight', 'loc_fc1.bias',
            'loc_fc2.weight', 'loc_fc2.bias']
    local_params=list(filter(lambda kv: kv[0] in local_list, model.named_parameters()))
    base_params=list(filter(lambda kv: kv[0] not in local_list, model.named_parameters()))

    local_params = [param[1] for param in local_params]
    base_params = [param[1] for param in base_params]
    
    optimizer = torch.optim.SGD([{'params':base_params},
                                {'params':local_params, 'lr':learning_rate/10}], 
                                lr=learning_rate, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=20, gamma=0.1
    )

    train_one_epoch(model, train_loader, loss_function, optimizer, epoch)
    val_acc_list.append(eval_one_epoch(model, test_loader, epoch,best_val_acc))
    best_val_acc = max(val_acc_list)
    scheduler.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3448: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


after looking at 128 images, running_loss is 2.1840
after looking at 6528 images, running_loss is 2.2349
after looking at 12928 images, running_loss is 2.2351
after looking at 19328 images, running_loss is 2.2372
after looking at 25728 images, running_loss is 2.2370
after looking at 32128 images, running_loss is 2.2391
after looking at 38528 images, running_loss is 2.2391
after looking at 44928 images, running_loss is 2.2387
after looking at 51328 images, running_loss is 2.2381
after looking at 57728 images, running_loss is 2.2382
after looking at 64128 images, running_loss is 2.2380
after looking at 70528 images, running_loss is 2.2378
epoch 0 : loss 2.2383
after looking at 32 images, running_acc is 0.2500
after looking at 1632 images, running_acc is 0.1875
after looking at 3232 images, running_acc is 0.1946
after looking at 4832 images, running_acc is 0.1966
after looking at 6432 images, running_acc is 0.1942
after looking at 8032 images, running_acc is 0.1926
after looking at 9632 i

KeyboardInterrupt: ignored